In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project


In [35]:
plate_number = 420
plate = get_postion_number(plate_number)
plate = 1
print(plate)
directory = directory_scratch
# directory = directory_project
listdir = os.listdir(directory)
list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
dates_datetime = get_dates_datetime(directory,plate)
len(list_dir_interest)

1


36

In [4]:
plate_number = get_plate_number(plate,dates_datetime[0])
print(0,plate_number)
for i in range(len(list_dir_interest)):
    new_plate_number = get_plate_number(plate,dates_datetime[i])
    if plate_number!=new_plate_number:
        plate_number=new_plate_number
        print(i,plate_number)


0 None


In [36]:
begin = 0
end = begin + 35

print('begin =', dates_datetime[begin], '\n  end =', dates_datetime[end])

begin = 2021-03-15 15:31:00 
  end = 2021-03-18 13:31:00


In [28]:
dates_datetime[72:75]

[datetime.datetime(2020, 10, 15, 19, 9),
 datetime.datetime(2020, 10, 15, 23, 9),
 datetime.datetime(2020, 10, 16, 3, 9)]

In [17]:
dates_datetime[begin:end]

[datetime.datetime(2020, 8, 14, 1, 36),
 datetime.datetime(2020, 8, 14, 5, 36),
 datetime.datetime(2020, 8, 14, 9, 36),
 datetime.datetime(2020, 8, 14, 13, 36),
 datetime.datetime(2020, 8, 14, 17, 36),
 datetime.datetime(2020, 8, 14, 21, 36),
 datetime.datetime(2020, 8, 15, 1, 36),
 datetime.datetime(2020, 8, 15, 5, 36),
 datetime.datetime(2020, 8, 15, 9, 36),
 datetime.datetime(2020, 8, 15, 13, 36),
 datetime.datetime(2020, 8, 15, 17, 37),
 datetime.datetime(2020, 8, 15, 21, 36),
 datetime.datetime(2020, 8, 16, 1, 36),
 datetime.datetime(2020, 8, 16, 5, 36),
 datetime.datetime(2020, 8, 16, 9, 36),
 datetime.datetime(2020, 8, 16, 13, 36),
 datetime.datetime(2020, 8, 16, 17, 37),
 datetime.datetime(2020, 8, 16, 21, 37)]

In [18]:
find_state(plate, begin, end, directory,True)

'skeletonization is complete'

In [39]:
find_state_extract(plate, begin, end, directory)

'extration is complete'

In [6]:
get_dirname(dates_datetime[14], plate)

'20200921_0805_Plate10'

In [33]:
start = begin + 34
finish = start + 2
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [34]:
plt.close('all')
start = 0
finish = start + 1
for i in range(start,finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
start = begin + 34
finish = start +2
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
    skel = skel_info['skeleton']
#     skels.append(skel)
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)
plt.close('all')
start = 0
finish = start + 1
for i in range(start, finish):
    plot_t_tp1([],[],None,None,skels[i],skels[i+1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Create graphs***

In [13]:
num_parallel = 20
time = '30:00'
args=[plate, directory]
run_parallel('extract_nx_graph.py',args,begin,end,num_parallel,time,'extract_nx')

***Extract Width***

In [15]:
num_parallel = 16
time = '1:00:00'
args = [plate,directory]
run_parallel('extract_width.py',args,begin,end,num_parallel,time,'extract_width')

***Identify Nodes***

In [38]:
num_parallel = 1
time = '12:00:00'
args=[plate,begin,end, directory]
run_parallel('extract_nodes_surf.py',args,0,0,num_parallel,time,'node_id')

***Hyphae extraction***

In [40]:
num_parallel = 1
time = '2:00:00'
args = [plate, begin, end, directory]
run_parallel('hyphae_extraction.py',args,0,0,num_parallel,time,'hyphae')